In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import pearsonr
import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Embedding,Input,Concatenate,BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.regularizers import Regularizer
from keras.layers.core import Reshape,Dropout,Dense
from keras.layers.embeddings import Embedding
import keras.backend as K

import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

pd.options.display.max_columns=150
pd.options.display.max_seq_items=150
pd.options.display.max_rows=150

In [ ]:
df_2014 = pd.read_csv('NBA_sched_2013-2014.csv')
df_2014['season']=2014

df_2014['Date'] = pd.to_datetime(df_2014['Date'])

a = df_2014['Date'].min()
df_2014['week']=0
for i in range(0,25):
    b = (a+pd.DateOffset(weeks=i))
    df_2014['week'] = np.where(df_2014['Date']>b,i+1,df_2014['week'])

df_2015 = pd.read_csv('NBA_sched_2014-2015.csv')
df_2015['Date'] = pd.to_datetime(df_2015['Date'])
df_2015['season']=2015
a = df_2015['Date'].min()
df_2015['week']=0
for i in range(0,25):
    b = (a+pd.DateOffset(weeks=i))
    df_2015['week'] = np.where(df_2015['Date']>b,i+1,df_2015['week'])

df_2016 = pd.read_csv('NBA_sched_2015-2016.csv')
df_2016['Date'] = pd.to_datetime(df_2016['Date'])
df_2016['season']=2016
a = df_2016['Date'].min()
df_2016['week']=0
for i in range(0,25):
    b = (a+pd.DateOffset(weeks=i))
    df_2016['week'] = np.where(df_2016['Date']>b,i+1,df_2016['week'])

df_2017 = pd.read_csv('NBA_sched_2016-2017.csv')
df_2017['Date'] = pd.to_datetime(df_2017['Date'])
df_2017['season']=2017
a = df_2017['Date'].min()
df_2017['week']=0
for i in range(0,25):
    b = (a+pd.DateOffset(weeks=i))
    df_2017['week'] = np.where(df_2017['Date']>b,i+1,df_2017['week'])

df_2018 = pd.read_csv('NBA_sched_2017-2018.csv')
df_2018['Date'] = pd.to_datetime(df_2018['Date'])
df_2018['season']=2018
a = df_2018['Date'].min()
df_2018['week']=0
for i in range(0,25):
    b = (a+pd.DateOffset(weeks=i))
    df_2018['week'] = np.where(df_2018['Date']>b,i+1,df_2018['week'])


df_full = pd.concat([df_2014,df_2015,df_2016,df_2017,df_2018])
df_full.head()

In [ ]:
a= pd.read_csv('NBA_Team_GameLogs_2014.csv')
b= pd.read_csv('NBA_Team_GameLogs_2015.csv')
c= pd.read_csv('NBA_Team_GameLogs_2016.csv')
d= pd.read_csv('NBA_Team_GameLogs_2017.csv')
e= pd.read_csv('NBA_Team_GameLogs_2018.csv')

df_gamelogs = pd.concat([a,b,c,d,e])
df_gamelogs.head()

In [ ]:
np.sort(df_full['Team'].unique())

In [ ]:
np.sort(df_gamelogs['Team'].unique())

In [ ]:
df_gamelogs['Team'] = np.where(df_gamelogs['Team']=='Oklahoma City',
                                    'Okla City',df_gamelogs['Team'])

df_gamelogs['Team'] = np.where(df_gamelogs['Team']=='Sacrametno',
                                    'Sacramento',df_gamelogs['Team'])


In [ ]:
df_gamelogs['date'] = pd.to_datetime(df_gamelogs['date'])

In [ ]:
df_gamelogs.rename(columns={'date':'Date'},inplace=True)

df_gamelogs.columns

In [ ]:
df_gamelogs = df_gamelogs[['Team', 'Date', 'homehelp', 'Tm', 'Opp.1', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF']]

In [ ]:
df1 = df_full.merge(df_gamelogs,how='left',on=['Team','Date'])
#df1.to_csv('full_sched.csv',index=False)
df1.columns

In [ ]:
np.sort(df1['Team'].unique())==np.sort(df1['Opp'].unique())

In [ ]:
df1['total'] = df1['TeamScore']+df1['OppScore']

#df1['actual_ou'] = np.where((df1['TeamScore']+df1['OppScore'])>df1['O/U'],1,
#               np.where((df1['TeamScore']+df1['OppScore'])<df1['O/U'],0,"p"))

df1['actual_ou'] = np.where((df1['TeamScore']+df1['OppScore'])>df1['O/U'],1,
                           np.where((df1['TeamScore']+df1['OppScore'])<df1['O/U'],0,"p"))

#df1['actual_ou'] = np.where((df1['TeamScore']+df1['OppScore'])>df1['O/U'],1,0)

df1['actual_spread_winner'] = np.where((df1['OppScore']-df1['TeamScore'])<df1['Line'],1,
                                       np.where((df1['OppScore']-df1['TeamScore'])>df1['Line'],0,"p"))

#df1 = df1[df1['actual_ou']!="p"]
#df1['actual_ou']=df1['actual_ou'].astype(float)
#df1.reset_index(drop=True,inplace=True)

le = LabelEncoder()
df1['tm_id'] = le.fit_transform(df1['Team'])
df1['opp_id'] = le.transform(df1['Opp'])


df1.head()

In [ ]:
df1.columns

#### Clean deep MF

In [ ]:
years = [2014,2015,2016,2017,2018]
weeks = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

df_wk = df1[(df1['season']==2015)&(df1['week']<=7)]
cols = df_wk.columns
cols = list(cols)
cols.append("est_fg")
cols.append("est_fga")
cols.append("est_3p")
cols.append("est_fta")
cols.append("est_orb")
cols.append("est_trb")
cols.append("est_ast")
cols.append("est_blk")
cols.append("est_tov")
cols.append("est_pf")
cols.append("est_score")

df_full_estimates = pd.DataFrame(columns=cols)

for yr in years:
    for wk in weeks:

        df_wk = df1[(df1['season']==yr)&(df1['week']<=wk)]
        
        
        df_wk.reset_index(inplace=True,drop=True)
                
        df_next = df1[(df1['season']==yr)&(df1['week']==wk+1)]
        
        tm_id_input = Input(shape=[1],name="tm")
        opp_id_input = Input(shape=[1],name="opp")
        home_input = Input(shape=[1],name="home")
        #fatigue_input = Input(shape=[4],name="fatigue")
        ou_input = Input(shape=[1],name="ou")
        line_input = Input(shape=[1],name="line")
        
        embedding_size=15
        
        tm_score_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_score_embedding")(tm_id_input)
        
        opp_score_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_score_embedding")(opp_id_input)
        
        tm_fg_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_fg_embedding")(tm_id_input)
        
        opp_fg_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_fg_embedding")(opp_id_input)
        
        tm_fga_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_fga_embedding")(tm_id_input)
        
        opp_fga_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_fga_embedding")(opp_id_input)
        
        tm_3p_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_3p_embedding")(tm_id_input)
        
        opp_3p_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_3p_embedding")(opp_id_input)
        
        tm_fta_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_fta_embedding")(tm_id_input)
        
        opp_fta_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_fta_embedding")(opp_id_input)
        
        tm_orb_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_orb_embedding")(tm_id_input)
        
        opp_orb_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_orb_embedding")(opp_id_input)
        
        tm_trb_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_trb_embedding")(tm_id_input)
        
        opp_trb_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_trb_embedding")(opp_id_input)
        
        tm_ast_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_ast_embedding")(tm_id_input)
        
        opp_ast_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_ast_embedding")(opp_id_input)
        
        tm_blk_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_blk_embedding")(tm_id_input)
        
        opp_blk_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_blk_embedding")(opp_id_input)
        
        tm_tov_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_tov_embedding")(tm_id_input)
        
        opp_tov_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_tov_embedding")(opp_id_input)
        
        tm_pf_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="tm_pf_embedding")(tm_id_input)
        
        opp_pf_embedding = Embedding(output_dim=embedding_size,input_dim=df_wk.shape[0],
                                input_length=1, name="opp_pf_embedding")(opp_id_input)
        
        tm_score_vecs = Reshape([embedding_size])(tm_score_embedding)
        opp_score_vecs = Reshape([embedding_size])(opp_score_embedding)
        
        tm_fg_vecs = Reshape([embedding_size])(tm_fg_embedding)
        opp_fg_vecs = Reshape([embedding_size])(opp_fg_embedding)
        
        tm_fga_vecs = Reshape([embedding_size])(tm_fga_embedding)
        opp_fga_vecs = Reshape([embedding_size])(opp_fga_embedding)
        
        tm_3p_vecs = Reshape([embedding_size])(tm_3p_embedding)
        opp_3p_vecs = Reshape([embedding_size])(opp_3p_embedding)
        
        tm_fta_vecs = Reshape([embedding_size])(tm_fta_embedding)
        opp_fta_vecs = Reshape([embedding_size])(opp_fta_embedding)
        
        tm_orb_vecs = Reshape([embedding_size])(tm_orb_embedding)
        opp_orb_vecs = Reshape([embedding_size])(opp_orb_embedding)
        
        tm_trb_vecs = Reshape([embedding_size])(tm_trb_embedding)
        opp_trb_vecs = Reshape([embedding_size])(opp_trb_embedding)
        
        tm_ast_vecs = Reshape([embedding_size])(tm_ast_embedding)
        opp_ast_vecs = Reshape([embedding_size])(opp_ast_embedding)
        
        tm_blk_vecs = Reshape([embedding_size])(tm_blk_embedding)
        opp_blk_vecs = Reshape([embedding_size])(opp_blk_embedding)
        
        tm_tov_vecs = Reshape([embedding_size])(tm_tov_embedding)
        opp_tov_vecs = Reshape([embedding_size])(opp_tov_embedding)
        
        tm_pf_vecs = Reshape([embedding_size])(tm_pf_embedding)
        opp_pf_vecs = Reshape([embedding_size])(opp_pf_embedding)
        
        
        input_fg_vecs = Concatenate()([tm_fg_vecs,opp_fg_vecs,home_input,ou_input,line_input])
        input_fga_vecs = Concatenate()([tm_fga_vecs,opp_fga_vecs])
        input_3p_vecs = Concatenate()([tm_3p_vecs,opp_3p_vecs,home_input,ou_input,line_input])
        input_fta_vecs = Concatenate()([tm_fta_vecs,opp_fta_vecs])
        input_orb_vecs = Concatenate()([tm_orb_vecs,opp_orb_vecs])
        input_trb_vecs = Concatenate()([tm_trb_vecs,opp_trb_vecs])
        input_ast_vecs = Concatenate()([tm_ast_vecs,opp_ast_vecs,home_input,ou_input,line_input])
        input_blk_vecs = Concatenate()([tm_blk_vecs,opp_blk_vecs])
        input_tov_vecs = Concatenate()([tm_tov_vecs,opp_tov_vecs])
        input_pf_vecs = Concatenate()([tm_pf_vecs,opp_pf_vecs])
        
        #x = BatchNormalization()(input_shots_vecs)
        x = Dense(64,activation='relu')(input_fg_vecs)
        fg = Dense(1,name="fg_output")(x)
        
        #x = BatchNormalization()(input_pim_vecs)
        x = Dense(64,activation='relu')(input_fga_vecs)
        fga = Dense(1,name="fga_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_3p_vecs)
        three_p = Dense(1,name="3p_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_fta_vecs)
        fta = Dense(1,name="fta_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_orb_vecs)
        orb = Dense(1,name="orb_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_trb_vecs)
        trb = Dense(1,name="trb_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_ast_vecs)
        ast = Dense(1,name="ast_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_blk_vecs)
        blk = Dense(1,name="blk_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_tov_vecs)
        tov = Dense(1,name="tov_output")(x)
        
        #x = BatchNormalization()(input_ppg_vecs)
        x = Dense(64,activation='relu')(input_pf_vecs)
        pf = Dense(1,name="pf_output")(x)
        
        input_score_vecs = Concatenate()([tm_score_vecs,opp_score_vecs,
                                         home_input,ou_input,line_input])
        x=BatchNormalization()(input_score_vecs)
        x = Dense(64,activation='relu')(x)
        score = Dense(1,name="score_output")(x)
        
        model = Model(inputs=[tm_id_input,opp_id_input,home_input,ou_input,line_input],
                      outputs=[fg,fga,three_p,fta,orb,trb,ast,blk,tov,pf,score])
        
        model.compile(optimizer="adam", loss="mse")
        
        #earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto')
        
        #lrplateau = ReduceLROnPlateau(patience=5)

        history = model.fit(x=[df_wk['tm_id'],df_wk['opp_id'],df_wk['Home?'],df_wk['O/U'],df_wk['Line']],
                   y=[df_wk['FG'],df_wk['FGA'],df_wk['3P'],
                     df_wk['FTA'],df_wk['ORB'],df_wk['TRB'],df_wk['AST'],df_wk['BLK'],
                     df_wk['TOV'],df_wk['PF'],df_wk['Tm']],
                   epochs=5,validation_split=0.1,batch_size=16,verbose=0)#,callbacks=[lrplateau])
        
        
        pred = model.predict([df_next['tm_id'],df_next['opp_id'],
                              df_next['Home?'],df_next['O/U'],df_next['Line']])
        a = pd.DataFrame(pred[0])
        b = pd.DataFrame(pred[1])
        c = pd.DataFrame(pred[2])
        d = pd.DataFrame(pred[3])
        e = pd.DataFrame(pred[4])
        f = pd.DataFrame(pred[5])
        g = pd.DataFrame(pred[6])
        h = pd.DataFrame(pred[7])
        i = pd.DataFrame(pred[8])
        j = pd.DataFrame(pred[9])
        k = pd.DataFrame(pred[10])
        
        pred = pd.concat([a,b,c,d,e,f,g,h,i,j,k],axis=1,)
        pred.columns=['est_fg','est_fga','est_3p','est_fta',
               'est_orb', 'est_trb','est_ast','est_blk','est_tov','est_pf','est_score']
        
        
        df_next.reset_index(inplace=True,drop=True)
        df_next['est_fg'] = pred['est_fg']
        df_next['est_fga'] = pred['est_fga']
        df_next['est_3p'] = pred['est_3p']
        df_next['est_fta'] = pred['est_fta']
        df_next['est_orb'] = pred['est_orb']
        df_next['est_trb'] = pred['est_trb']
        df_next['est_ast'] = pred['est_ast']
        df_next['est_blk'] = pred['est_blk']
        df_next['est_tov'] = pred['est_tov']
        df_next['est_pf'] = pred['est_pf']
        df_next['est_score'] = pred['est_score']
        
        print("\n\n Week: "+ str(wk)+"\n\n")
        
        print(pearsonr(df_next['est_fg'],df_next['FG']))
        
        print(pearsonr(df_next['est_3p'],df_next['3P']))
        
        print(pearsonr(df_next['est_trb'],df_next['TRB']))
        
        print(pearsonr(df_next['est_tov'],df_next['TOV']))
        
        print(pearsonr(df_next['est_score'],df_next['Tm']))
        
        df_full_estimates = pd.concat([df_full_estimates,df_next])
        
        try:
            if wk==4:
                df_full_estimates.to_csv('test.csv',index=False)
        except:
            continue
            
        del model
        K.clear_session()
        
df_full_estimates.to_csv('df_full_estimates.csv',index=False)